
# **Graded Notebook 2025**

#### **Dataset information**
We will use the **Health Insurance Costs** dataset (`insurance.csv`). Each row describes a person's demographic information (age, sex, BMI, number of children, region, smoker status) and the medical charges billed by an insurance company.

Download the dataset here: [insurance.csv](https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv) and save it in the same directory as this notebook.

**Focus:** Scoring prioritizes how well you *interpret* and communicate the results (plots, coefficients, errors, and caveats). Raw accuracy is secondary. Use markdown cells to explain what the numbers mean in plain language.



---
## **Task 1: Explore and prepare the data (15 points)**

#### **1.1: Load and inspect (5 points)**
- Load `insurance.csv` and display the head, shape, and data types.
- Identify the meaning of each column and which are categorical vs. numerical, note any immediate quirks you observe (e.g., units, obvious outliers).
- Which column is likely to be you target? Why?


In [18]:
# Task 1.1



#### **1.2: Understand distributions and relationships (10 points)**
- Visualize distributions and explore relationships between the features and the target.
- Summarize 2–3 hypotheses about which factors likely drive costs and why.

In [17]:
# Task 1.2



---
## **Task 2: Regression – Predict medical charges (30 points)**

#### **2.1: Baseline linear models (15 points)**
- Using the features of your choice (likely linked to your hypothesis above), build a linear regression model to predict your target. Be mindful about the encoding of categorical values.
- Evaluate with at least two metrics and interpret their individual meaning.
- Interpret the sign and relative magnitude of coefficients linked to each feature in the model: what do they suggest about the link between each feature and the target?


In [ ]:
# Task 2.1



#### **2.2: Improving metrics (15 points)**
- Try improving your results, one can chose one or more of the following methods:
    - Adding or removing some features
    - Transforming some features (e.g. polynomials)
    - Using another regression algorithm
- Are your results better? Why?


In [ ]:
# Task 2.2



---
## **Task 3: Classification – Identify high-cost patients (30 points)**

#### **3.1: Define a binary outcome (5 points)**
- Create a `high_cost` label using a clear threshold on the charges column. Justify the cutoff and show the class balance.


In [ ]:
# Task 3.1


#### **3.2: Features analysis (10 points)**
- Similar to 1.2, explore the relationship between the features and this new target. What can you say?

In [ ]:
# Task 3.2



#### **3.3: Train and compare classifiers (15 points)**
- Train at least two different classifiers algorithms with sensible preprocessing.
- Evaluate the results with a set of classifiaction metrics, justify your choices and interpret their meaning.
- Interpret the types of mistakes each model makes and which features drive their decisions.


In [ ]:
# Task 3.3



---
## **Task 4: PCA (15 points)**
- Apply a PCA to reduce the number of features. 
- Carefully select the number of PC to use and document this choice.
- Train a model on these new features and iscuss how PCA affects model performances.

In [ ]:
# Task 4



---
## **Task 5: Unsupervised insights (10 points)**
- Apply clustering (e.g., K-Means) or another unsupervised method on selected features.
- Visualize and describe the discovered profiles: what characterizes each cluster?


In [21]:
# Task 5



---
## **Bonus: Anything more? (Optional)**
- If you feel like, implement anything that was not covered in this notebook (e.g. another model or task, cross validation, ...)

In [22]:
# Bonus Task
